In [1]:
from gssutils import *
import pandas as pd
import json

#extract spread sheet from landing page
scraper = Scraper(seed="info.json")
scraper.distributions = [x for x in scraper.distributions if hasattr(x, "mediaType")]
scraper

# Add cubes class
cubes = Cubes("info.json")
#Add tracer to transform
trace = TransformTrace()

In [2]:
# extract latest distribution and datasetTitle
distribution = scraper.distribution(latest = True)
datasetTitle = distribution.title
print(distribution.downloadURL)
print(datasetTitle)

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/861819/December_2019_Sub-regional_Feed-in_Tariffs_confirmed_CFR.xls
Sub-regional Feed-in Tariffs confirmed on the CFR statistics


In [3]:
# Extract all the tabs from the spread sheet
tabs = {tab.name: tab for tab in distribution.as_databaker()}

In [4]:
# List out all the tab name to cross verify with the spread sheet
for tab in tabs:
    print(tab)

Title
Latest Quarter - Region
Latest Quarter - Region (kW)
Latest Quarter - LA
Latest Quarter - LA (kW)
Latest Quarter - PC
Latest Quarter - PC (kW)
Latest Quarter - LEPs
Latest Quarter - LEPs (kW)
Calculation


In [5]:
columns = ["Region", "Period", "Technology", "Installation", "Households", "Local Or Parliamentary Code",
           "Local Enterprise Partnerships", "Leps Authority", "Marker", "Unit"]

In [6]:
# Filtering out the tabs which are not required and start the transform 
for name, tab in tabs.items():
    if 'Title' in name or 'Calculation' in name \
    or'Latest Quarter - LA' in name or 'Latest Quarter - LA (kW)' in name \
    or 'Latest Quarter - PC' in name or 'Latest Quarter - PC (kW)' in name \
    or 'Latest Quarter - LEPs' in name or 'Latest Quarter - LEPs (kW)' in name:
        continue
    print(tab.name)
    trace.start(datasetTitle, tab, columns, distribution.downloadURL)
        
    cell = tab.excel_ref("B7")
    
    footer = tab.filter(contains_string("Notes")).expand(RIGHT).expand(DOWN)
    
    region = cell.fill(DOWN).is_not_blank().is_not_whitespace()-footer
    trace.Region("Taken from cell B7 down excluding footer")
        
    households = cell.fill(RIGHT).is_not_blank().is_not_whitespace()
    trace.Households("Taken from cell B7 right")
        
    technology = cell.shift(0, -1).fill(RIGHT).is_not_blank().is_not_whitespace()
    trace.Technology("Taken from cell B6 right which is not blank")

    #installation may potentially become measure type. A word from DM is awaited.    
    installation = cell.shift(0, -2).fill(RIGHT).is_not_blank().is_not_whitespace()
    trace.Installation("Taken from cell B5 right which is not blank")

    period = cell.shift(0, -4).fill(RIGHT).is_not_blank().is_not_whitespace()
    trace.Period("taken from cell B3 right which is not blank")

    observations = region.fill(RIGHT).is_not_blank().is_not_whitespace()-footer

    dimensions = [
        HDim(region, "Region", DIRECTLY, LEFT),
        HDim(households, "Households", CLOSEST, LEFT),
        HDim(technology, "Technology", CLOSEST, LEFT),
        HDim(installation, "Installation", CLOSEST, LEFT),
        HDim(period, "Period", CLOSEST, LEFT)
    ]
    tidy_sheet = ConversionSegment(tab, dimensions, observations)
    savepreviewhtml(tidy_sheet,fname=tab.name + "Preview.html")
    trace.with_preview(tidy_sheet)
    trace.store("combined_dataframe", tidy_sheet.topandas())

Latest Quarter - Region
tablepart 'Latest Quarter - Region' written #injblock1001
javascript calculated


/Users/santhoshthangavel/projects/family-edvp/datasets/BEIS-Sub-regional-Feed-in-Tariffs-confirmed-on-the-CFR-statistics/Latest Quarter - RegionPreview.html

tablepart 'Latest Quarter - Region' written #injblock1002
javascript calculated


/Users/santhoshthangavel/projects/family-edvp/datasets/BEIS-Sub-regional-Feed-in-Tariffs-confirmed-on-the-CFR-statistics/ecd73174-6d50-4e9e-8f52-219de381084d.html


Latest Quarter - Region (kW)
tablepart 'Latest Quarter - Region (kW)' written #injblock1003
javascript calculated


/Users/santhoshthangavel/projects/family-edvp/datasets/BEIS-Sub-regional-Feed-in-Tariffs-confirmed-on-the-CFR-statistics/Latest Quarter - Region (kW)Preview.html

tablepart 'Latest Quarter - Region (kW)' written #injblock1004
javascript calculated


/Users/santhoshthangavel/projects/family-edvp/datasets/BEIS-Sub-regional-Feed-in-Tariffs-confirmed-on-the-CFR-statistics/707ed9e9-77db-4b57-979a-2f58e584f4df.html

In [7]:
for name, tab in tabs.items():
    if 'Title' in name or 'Calculation' in name \
    or 'Latest Quarter - Region' in name or 'Latest Quarter - Region (kW)' in name \
    or 'Latest Quarter - LEPs' in name or 'Latest Quarter - LEPs (kW)' in name:
        continue
    print(tab.name)
    trace.start(datasetTitle, tab, columns, distribution.downloadURL)   
    cell = tab.excel_ref("B7")
    
# Datamarker is catching footer values from Latest Quarter - LA and Latest Quarter - LA (kW) tabs
    footer = tab.filter(contains_string("Notes")).expand(RIGHT).expand(DOWN)
    
    #datamarker is catching weired values from footer so footer is caught and deleted    
    local_or_parliamentary_code = cell.fill(DOWN).is_not_blank().is_not_whitespace()-footer
    trace.Local_Or_Parliamentary_Code("Taken from cell B7 down excluding footer")

    households = cell.shift(1, 0).fill(RIGHT).is_not_blank().is_not_whitespace()
    trace.Households("Taken from cell B7 right")
        
    technology = cell.shift (0, -1).fill(RIGHT).is_not_blank().is_not_whitespace()
    trace.Technology("Taken from cell B6 right which is not blank")
    
    #installation may potentially become measure type. A word from DM is awaited.
    installation = cell.shift (0, -2).fill(RIGHT).is_not_blank().is_not_whitespace()
    trace.Installation("Taken from cell B5 right which is not blank")

    period = cell.shift(0, -4).fill(RIGHT).is_not_blank().is_not_whitespace()
    trace.Period("taken from cell B3 right which is not blank")

    #datamarker is catching weired values from footer so footer is caught and deleted
    observations = households.fill(DOWN).is_not_blank().is_not_whitespace()-footer

    dimensions = [
        HDim(local_or_parliamentary_code, "Local Or Parliamentary Code", CLOSEST, ABOVE),
        HDim(households, "Households", CLOSEST, LEFT),
        HDim(technology, "Technology", CLOSEST, LEFT),
        HDim(installation, "Installation", CLOSEST, LEFT),
        HDim(period, "Period", CLOSEST, LEFT)
    ]
    tidy_sheet = ConversionSegment(tab, dimensions, observations)
    savepreviewhtml(tidy_sheet,fname=tab.name + "Preview.html")
    trace.with_preview(tidy_sheet)
    trace.store("combined_dataframe", tidy_sheet.topandas())

# # changes in local authority code to be implemented in post processing
# # changes in local authority name to be implemented in post processing

Latest Quarter - LA
tablepart 'Latest Quarter - LA' written #injblock1005
javascript calculated


/Users/santhoshthangavel/projects/family-edvp/datasets/BEIS-Sub-regional-Feed-in-Tariffs-confirmed-on-the-CFR-statistics/Latest Quarter - LAPreview.html

tablepart 'Latest Quarter - LA' written #injblock1006
javascript calculated


/Users/santhoshthangavel/projects/family-edvp/datasets/BEIS-Sub-regional-Feed-in-Tariffs-confirmed-on-the-CFR-statistics/349a65d6-3339-41bc-bcfa-e7659740eb66.html


Latest Quarter - LA (kW)
tablepart 'Latest Quarter - LA (kW)' written #injblock1007
javascript calculated


...

/Users/santhoshthangavel/projects/family-edvp/datasets/BEIS-Sub-regional-Feed-in-Tariffs-confirmed-on-the-CFR-statistics/db3e615b-6ce5-43cb-8c91-ae303016cde2.html


Latest Quarter - PC (kW)
tablepart 'Latest Quarter - PC (kW)' written #injblock1011
javascript calculated


/Users/santhoshthangavel/projects/family-edvp/datasets/BEIS-Sub-regional-Feed-in-Tariffs-confirmed-on-the-CFR-statistics/Latest Quarter - PC (kW)Preview.html

tablepart 'Latest Quarter - PC (kW)' written #injblock1012
javascript calculated


/Users/santhoshthangavel/projects/family-edvp/datasets/BEIS-Sub-regional-Feed-in-Tariffs-confirmed-on-the-CFR-statistics/f762261b-3afb-48ce-9144-aa9039fefafb.html

In [8]:
for name, tab in tabs.items():
    if 'Title' in name or 'Calculation' in name \
    or 'Latest Quarter - Region' in name or 'Latest Quarter - Region (kW)' in name \
    or 'Latest Quarter - LA' in name or 'Latest Quarter - LA (KW)' in name \
    or 'Latest Quarter - PC' in name or 'Latest Quarter - PC (kW)' in name:
        continue
    print(tab.name)
    trace.start(datasetTitle, tab, columns, distribution.downloadURL)
        
    cell = tab.excel_ref("B7")
    
    footer = tab.filter(contains_string("Notes")).expand(RIGHT).expand(DOWN)
        
    local_enterprise_partnerships = cell.fill(DOWN).is_not_blank().is_not_whitespace()-footer
    trace.Local_Enterprise_Partnerships("Taken from cell B7 down excluding footer")

    leps_authority = cell.shift(1, 0).fill(DOWN).is_not_blank().is_not_whitespace()
    trace.Leps_Authority("Taken from cell C7 down")

    households = cell.shift(1, 0).fill(RIGHT).is_not_blank().is_not_whitespace()
    trace.Households("Taken from cell C7 right")
        
    technology = cell.shift (0, -1).fill(RIGHT).is_not_blank().is_not_whitespace()
    trace.Technology("Taken from cell B6 right which is not blank")

    #installation may potentially become measure type. A word from DM is awaited.
    installation = cell.shift (0, -2).fill(RIGHT).is_not_blank().is_not_whitespace()
    trace.Installation("Taken from cell B5 right which is not blank")

    period = cell.shift(0, -4).fill(RIGHT).is_not_blank().is_not_whitespace()
    trace.Period("taken from cell B3 right which is not blank")

    observations = leps_authority.fill(RIGHT).is_not_blank().is_not_whitespace()-footer

    dimensions = [
        HDim(local_enterprise_partnerships, "Local Enterprise Partnerships", CLOSEST, ABOVE),
        HDim(leps_authority, "Leps_Authority", CLOSEST, ABOVE),
        HDim(households, "Households", CLOSEST, LEFT),
        HDim(technology, "Technology", CLOSEST, LEFT),
        HDim(installation, "Installation", CLOSEST, LEFT),
        HDim(period, "Period", CLOSEST, LEFT)
    ]
    tidy_sheet = ConversionSegment(tab, dimensions, observations)
    savepreviewhtml(tidy_sheet,fname=tab.name + "Preview.html")
    trace.with_preview(tidy_sheet)
    trace.store("combined_dataframe", tidy_sheet.topandas())

Latest Quarter - LEPs
tablepart 'Latest Quarter - LEPs' written #injblock1013
javascript calculated


/Users/santhoshthangavel/projects/family-edvp/datasets/BEIS-Sub-regional-Feed-in-Tariffs-confirmed-on-the-CFR-statistics/Latest Quarter - LEPsPreview.html

tablepart 'Latest Quarter - LEPs' written #injblock1014
javascript calculated


/Users/santhoshthangavel/projects/family-edvp/datasets/BEIS-Sub-regional-Feed-in-Tariffs-confirmed-on-the-CFR-statistics/3b76c5fa-73fb-4b5d-95de-f4666beed8a1.html


Latest Quarter - LEPs (kW)
tablepart 'Latest Quarter - LEPs (kW)' written #injblock1015
javascript calculated


/Users/santhoshthangavel/projects/family-edvp/datasets/BEIS-Sub-regional-Feed-in-Tariffs-confirmed-on-the-CFR-statistics/Latest Quarter - LEPs (kW)Preview.html

tablepart 'Latest Quarter - LEPs (kW)' written #injblock1016
javascript calculated


/Users/santhoshthangavel/projects/family-edvp/datasets/BEIS-Sub-regional-Feed-in-Tariffs-confirmed-on-the-CFR-statistics/b7fa7c2d-00c7-40ed-bb4e-9f7d786b345b.html

In [9]:
df = trace.combine_and_trace(datasetTitle, "combined_dataframe")
df

,OBS,Region,Households,Technology,Installation,Period,DATAMARKER,Local Or Parliamentary Code,Local Enterprise Partnerships,Leps_Authority
0,2.03685e+06,East Midlands,Estimated number of households3,None,None,None,NaN,NaN,NaN,NaN
1,84388,East Midlands,Domestic,Photovoltaics,Cumulative number of installations 2,2019.0,NaN,NaN,NaN,NaN
2,87401,East Midlands,Total,Photovoltaics,Cumulative number of installations 2,2019.0,NaN,NaN,NaN,NaN
3,167,East Midlands,Domestic,Wind,Cumulative number of installations 2,2019.0,NaN,NaN,NaN,NaN
4,385,East Midlands,Total,Wind,Cumulative number of installations 2,2019.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4584,4.95,NaN,Domestic,MicroCHP,Cumulative number of installations 2,2019.0,NaN,NaN,York & North Yorkshire,York
4585,4.95,NaN,Total,MicroCHP,Cumulative number of installations 2,2019.0,NaN,NaN,York & North Yorkshire,York
4586,10558.2,NaN,Total Domestic,MicroCHP,Cumulative number of installations 2,2019.0,NaN,NaN,York & North Yorkshire,York
4587,1300.26,NaN,Total Non-Domestic,MicroCHP,Cumulative number of installations 2,2019.0,NaN,NaN,York & North Yorkshire,York


In [10]:
df['DATAMARKER'].unique()

array([nan, '-'], dtype=object)

In [11]:
cubes.add_cube(scraper, df.drop_duplicates(), datasetTitle)
cubes.output_all()

In [12]:
trace.render("spec_v1.html")

Template https://raw.githubusercontent.com/GSS-Cogs/frontend-template-resources/master/templates/jinja2/spec_v1.html rendered as ./out/spec.html
